In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image
import pandas as pd
from mpl_toolkits.basemap import Basemap
from matplotlib import cm
import warnings
warnings.filterwarnings("ignore")

ImportError: No module named 'mpl_toolkits.basemap'

### Review of lecture 20


In Lecture 20 we:

- learned how to plot different map projections

- learned about some properties of a Basemap object

### In this lecture we will:

- Learn about gridding and contouring.





### Gridding

Remember this figure?   


In [ ]:
Image(filename='Figures/etopo20.jpg',width=500)

We now have the tools to make it!

First, you should recognize the basic orthographic projection from last lecture.

In [ ]:
# lon_0, lat_0 are the center point of the projection.
plt.figure(1,(5,5)) # make the figure instance with a size of 5x5
m = Basemap(projection='ortho',lon_0=-75,lat_0=42) # make an orthographic projection map object
m.drawcoastlines() # put on the coastlines
# draw parallels and meridians.
m.drawparallels(np.arange(-90.,120.,30.))
m.drawmeridians(np.arange(0.,420.,60.));


And remember the topography data in the Etopo20 files from the lecture on the hypsographic curve (Lecture 15):

In [ ]:
etopo=np.loadtxt('Datasets/Etopo/etopo20data.gz')
elons=np.loadtxt('Datasets/Etopo/etopo20lons.gz')
elats=np.loadtxt('Datasets/Etopo/etopo20lats.gz')
print (etopo.shape)
print (elons.shape)
print (elats.shape)



**elats** is a 1D array with 540 latitudinal bins (this is a long and skinny array- 540 x 1). 

**elons** is a 1D array with 1081 longitudinal bins (this as a fat and wide array- 1 x 1081).  
And **etopo** is a 2D array with 540 rows and 1081 columns (540 x 1080).  

So **etopo** has an elevation for each lat/lon cell.  

In order to plot the elevation data onto a lat/lon grid, we have to first make a grid out of the 1D arrays of elats/ and elons.  

We use the **numpy** function **meshgrid** for this.  


In [ ]:
help (np.meshgrid)

Let's try out meshgrid on some smaller arrays first, to wrap our mind around it, so to speak. 

In [ ]:
x = np.arange(-3.2, 3.3, 0.1) # make a 1D array from -3.3 to 3.2 with a spacing of .1
y = np.arange(-3.2, 3.3, 0.1) # ditto
xx, yy = np.meshgrid(x, y) # make a meshgrid
print (x.shape)
print (y.shape)
print (xx.shape)
print (xx[0:5])

Now let's put some 'data' on this grid:

In [ ]:
z = np.sin(xx**2 + yy**2) / (xx**2 + yy**2)
print (z.shape)
print (z[0:5])

### Contouring

One way to visualize data in a 2D grid is to make a contour plot of it.  For this we
use the **matplotlib.pyplot** function **contour**  or **contourf**.  

In [ ]:
help(plt.contourf)

In [ ]:
h = plt.contour(x,y,z,cmap=cm.jet) # plot the contours
plt.axis('equal'); 
# this makes the axes "square" so we get a circle and not a squashed ellipse


Just in case you want to label your contours, you can use the **clabel** function from **matplotlib.pyplot**. 

In [ ]:
fig = plt.contour(x,y,z,levels=np.arange(0,1.20,.20),cmap=cm.jet)
plt.axis('equal')
plt.clabel(fig, inline=1, fontsize=10,fmt='%3.2f');

Contour plots are  all well and good, but what we were after was a continuous gradation of color, not the contour lines, so for that we use the function **contourf** instead: 

In [ ]:
fig = plt.contourf(x,y,z,cmap=cm.jet)
plt.axis('equal') # this makes the axes square
plt.axis('off'); # and this turns off the frame, ticks and labels


So, where were we? Oh yes, we wanted to plot the contoured elevation data onto an orthographic projection.  First, we will generate a meshgrid using **np.meshgrid** from **numpy** and the method **contourf** of our  map object **m** defined above.  The method **m.contourf** works like **matplotlib**'s:

In [ ]:
xx, yy = np.meshgrid(elons,elats)
x,y=m(xx,yy)
cs=m.contourf(x,y,etopo,30,cmap=cm.jet) # the 30 is the number of contours


Tada!!!

### Application to the Earth's magnetic field

Now we know how to grid and contour data, we can try to plot other features of our planet, like the magnetic field strength or inclination (dip below or above the horizontal).  I prepared a module named **mkigrf** for your mapping pleasure.   It includes a few functions that evaluate the elements of the magnetic field for any date between 1900 and 2020 including total field strength, direction, and radial field strength. Up until 2015, the data come from the  International Geomagnetic Reference Field (IGRF) model and after that, the data are extrapolated.  To learn more, check out this website:  https://www.ngdc.noaa.gov/IAGA/vmod/igrf.html.

You can use the function **mkigrf.doigrf** to find out the magnetic declination at a particular time and place. You could then use this function to set your compass.  Handy for us geology types...  

But  first, I should explain what the magnetic vector is.  As with all vectors, it has both direction and length.  We can express the vector in terms of cartesian coordinates (say, North, East and Down) or these polar coordinates:
- declination: the angle of the magnetic direction in the horizontal plane with respect to the North pole
- inclination: the angle of the magnetic direction in the vertical plane with respect to the horizontal
- intensity: the strength of the field, usually in units of tesla (either nano or micro). Tesla is is magnetic induction and is usually represented by the letter **B**.  

In [ ]:
Image(filename='Figures/components.png')

_a) Lines of flux produced by  a geocentric axial dipole.  b) Lines of flux  of the geomagnetic field of 2005.   At point P the horizontal component of the  field_ $B_H$, _is directed toward the magnetic north.  The vertical component_ $B_V$ _is directed down and the field makes an angle_ $I$ _with the horizontal, known as the
inclination.  c) Components of the geomagnetic field vector_ ${\bf B}$.  _The angle between the horizontal component (directed toward magnetic north and geographic north is the 
declination_ $D$._) [Modified from Ben-Yosef et al., 2008, doi:10.1016/j.jas.2008.05.016] _

In [ ]:
import mkigrf
help(mkigrf.doigrf)

**mkigrf.doigrf** returns $x,y,z$ cartesian components of the magnetic field vector.  But we want to plot the polar coordinates _declination, inclination, and strength_.  So we need to convert from cartesian coordinates to polar coordinates.  There is a handy function  **mkigrf.cart2dir()** that will do this for you.   For example, we find the declination for San Diego in 2017 like this:

In [ ]:
San_lat=33
San_lon=243
x,y,z,f=mkigrf.doigrf(San_lon,San_lat,2018)
Dec,Inc,B=mkigrf.cart2dir(x,y,z)
print ('%7.1f'%(Dec)) # notice the formatting

We can use the tools in **mkigrf** to evaluate the magnetic field over the surface of the Earth and then contour it on, say, a Hammer projection.  I'm going to go ahead and make a function for this so we can change the map central longitude and date as we like. 

In [ ]:
def magMap(date,**kwargs):
    """
    generates the data for a map of the magnetic field. 
    Inputs: 
    required: 
        date = decimal year for evaluation (between 1900 and 2020)
    optional: 
        lon_0 = desired zero longitude
    
    Returns: 
    
    Bdec = declinations
    Binc = inclinations
    B = field strength (in microtesla)
    lons = array of longitudes
    lats = array of latitudes

    """
    if 'lon_0' in kwargs.keys(): # check if there are keyword arguments
        lon_0=kwargs['lon_0'] # if lon_0 is set, use that one
    else: # otherwise..... 
        lon_0=0. # set the default lon_0 to 0. 
    
    incr=10 # we can vary to the resolution of the model
    lonmax=(lon_0+180.)%360+incr # get some parameters for our arrays of lat/lon
    lonmin=(lon_0-180.)
    latmax=90+incr
    lons=np.arange(lonmin,lonmax,incr) # make a 1D array of longitudes (like elons)
    lats=np.arange(-90,latmax,incr)# make a 1D array of longitudes (like elats)
    
    
    # set up some containers for the field elements
    lenLats, lenLons = len(lats), len(lons)
    B=np.zeros((lenLats,lenLons))
    Binc=np.zeros((lenLats,lenLons))
    Bdec=np.zeros((lenLats,lenLons))
    Brad=np.zeros((lenLats,lenLons))
    
    for j in range(lenLats): # step through the latitudes
        for i in range(lenLons): # and the longitudes
            x,y,z,f=mkigrf.doigrf(lons[i],lats[j],date)  # get the field elements
            Dec,Inc,Int=mkigrf.cart2dir(x,y,z) # turn them into polar coordites
            B[j][i]=Int*1e-3 # convert the string to microtesla (from nT)
            Binc[j][i]=Inc # store the inclination value
            Bdec[j][i]=Dec # store the declination value
    return Bdec,Binc,B,lons,lats # return the arrays.  



We can call magMap for any date we like.  

In [ ]:
date=2018 # let's do this for 2018 (actually, this is the beginning of 2017)
lon_0=0 # we can specify the grid spacing and the intended 0 longitude for the plot
Ds,Is,Bs,lons,lats=magMap(date)

And.... drum roll ....  we can plot them on a contour map.  

In [ ]:
m = Basemap(projection='hammer',lon_0=lon_0)
xx, yy = np.meshgrid(lons,lats)
x,y=m(xx,yy)

m.drawcoastlines()
lincr=1
levmax=round(Bs.max())+lincr
levmin=round(Bs.min()-lincr)

cs=m.contourf(x,y,Bs,levels=np.arange(levmin,levmax,lincr),cmap=cm.jet) 
# by assigning the contourf object to cs, we can add a colorbar
cbar=m.colorbar(cs,location='bottom')
plt.title('Field strength ($\mu$T): '+str(date));



In [ ]:
levmin,levmax

Let's apply the same technique to map the inclinations.

In [ ]:
m = Basemap(projection='hammer',lon_0=lon_0)
xx,yy = np.meshgrid(lons, lats)
x,y=m(xx,yy)

m.drawcoastlines()
lincr=1
levmax=Is.max()+lincr
levmin=round(Is.min()-lincr)

cs=m.contourf(x,y,Is,levels=np.arange(levmin,levmax,lincr),cmap=cm.jet)
# put on contour lines at 10 degree intervals with  black lines
m.contour(x,y,Is,levels=np.arange(-80,90,10),colors='black') 
# by assigning the contourf object to cs, we can add a colorbar
cbar=m.colorbar(cs,location='bottom')
plt.title('Field inclination: '+str(date));



Okay - i love magnetic fields and I think these maps are cool. 

### Assignment #7

In 1600, William Gilbert hypothesized that the Earth itself was a giant bar magnetic and that this gave rise to the Earth's **magnetic field**!  If it were true- that the source of the magnetic field behaved like a giant bar magnet- then the **inclination** of the **magnetic field** would vary as a function of latitude. 

**Inclination** is the angle between the horizontal and the direction of the field.   If the field were generated by a bar magnet, then the **inclination** would be horizontal (0 $^\circ$) at the magnetic equator and vertical ($\pm 90 ^\circ$) at the North and South poles. The equation that relates **inclination** ($I$) and latitude ($\lambda$) is:

$$ \tan I = 2 \tan \lambda \quad \quad(dipole\  equation)$$        

7.1 Calculate the inclination at every latitude using the dipole equation:

-  Write a function that calculates inclination (use the dipole equation)

-  Apply the function you wrote to latitudes ranging from -90 to +90 at one degree intervals.  
    - [**Hint:** Remember that **np.tan( )**  and **np.arctan( )** work in radians and your plot should be in degrees.]   
    


7.2 Calculate the actual inclination at every latitude in your birth year:

- Use the function **magMap( )** to evaluate the magnetic field for the year you were born

- Use **meshgrid( )** to make a 2-D array of the latitudes and longitudes that were returned from **magMap( )**

- Transpose the new 2-D latitude array and then flatten the transposed array into a 1D array

- Transpose the inclination array that was returned by **magMap( )** and then flatten it into a 1D array.  

- Use the function **np.polyfit( )** to calculate a best fitting 3rd order polynomial through the data

- Use **np.polyval()** to evaluate the curve at all the latitudes

7.3 Compare the inclination that we expect from a bar magnet (calculated in 7.1) to the actual inclination in your birth year (calculated in 7.2) 

- Plot the latitudes and inclinations that you calculated in 7.1. Plot the data points as stars and label this line 'Ideal GAD'
    - [**Hint:** use * ]
        
- Plot the latitude against the inclincations you calculated in 7.2. Label the points with the year of your birth

- Plot the best fitting curve you evaluated in 7.2. Plot the points as black plus signs and label this line 'Best fit'
    - [**Hint:** use k+ ]

- Add a legend, title, x-axis, and y-axis to your figure

7.4 Make a contour map of the field strength in your birth year

- The map should be a hammer projection with lon_0 set to 0

- Add a title and colorbar to your figure